In [2]:
search_for = 135
start_from = 0
threads = 10
thread = 2
log_freq = 10000



#in this cell, for a given lattice, we compute all forbidden nodes, 
#i.e. those nodes whose Voronoi cell is at a distance less than doubled covering radius

#An* represented in m=n+1 as sum x_i=0 with x_i\equiv to each other mod m. 
#Forbidden nodes will be recorded w.r.t the basis (-n,1,1,..) and permutations

n=5 #lattice dim
m=n+1 #space dim




R = sqrt(n*(n+1)*(n+2)/12) #covering radius

#first we generate all nodes with norm <=4R, and with norm<=2R
t=1 #.3 #can increase a bit for random from non-forbidden
short = []
vshort = []
K = ceil(t*4*R)
K2 = (t*4*R)^2
Kv = (2*R)^2

indexes = [] #arrays of integers between -K and K having the same residue modulo m 
for rem in range(m):
    res = []
    for i in range(-K,K+1):
        if i%m==rem:
            res.append(i)
    indexes.append(res)

for c in range(0,K+1):
    for coefs in mrange_iter([indexes[c%m] for i in range(m-2)]):
        p = [c]+list(coefs)+[-c-sum(coefs)]
        is_sorted = True
        for i in range(m-1):
            if p[i]<p[i+1]:
                is_sorted = False
                break
        if is_sorted:
            norm2 = sum((p[i])^2 for i in range(m))
            if norm2>0:
                if norm2 <= Kv:
                    vshort.append(p)
                if norm2 <= K2:
                    short.append(p)           

print("very short:",len(vshort))
print("short:",len(short))


from time import time
tm = time()


#defining the cone arising due to symmetry - it suffices to construct Voronoi cell in it
ineqs = [] 
for i in range(m-1):
    toadd = [0]*(m+1)
    toadd[i+1]=1
    toadd[i+2]=-1
    ineqs.append(toadd) 
    
ineqs += [[0]+[1]*m,[0]+[-1]*m] #we are in the hyperplane where all coords sum to zero

#now Voronoi cell planes
for p in vshort: #suffices to look at nodes at most 2R
    ineqs.append([sum((p[i])^2 for i in range(m))]+[-2*p[i] for i in range(m)])

V = Polyhedron(ieqs=ineqs)
    
print(time()-tm)    

### orthogonal projection affine operator onto (improper) non-empty polytope 
zerom = matrix(m)
def projp(P):
    nv = P.n_vertices()
    z = vector(P.vertices()[0])
    if nv==1:
        return zerom,z
    A = matrix([vector(P.vertices()[i])-z for i in range(1,nv)]).transpose()
    return A*A.pseudoinverse(),z


forb = []

for d in range(0,n): #dim of face
    for ff in V.faces(d):
        proj = projp(ff)
        for p in short:
            if p not in forb:
                x = vector([t/2 for t in p]) #dist from 1/2 of a node to the Voronoi cell equals 1/2 of the dist between cells at zero and at the node
                proj_pnt = proj[0]*(x-proj[1])+proj[1]
                if proj_pnt in ff.as_polyhedron():
                    dist = (x-proj_pnt).norm()
                    if dist<R:
                        forb.append(p)

print("forbidden:",len(forb))

#expand by the basis (-n,1,1,..) and permutations
def basis_coefs(x):
    y = x[0:(m-1)]
    sy = sum(y)
    return [(-sy-y[i])/m for i in range(m-1)]

### forbd = forb with possible permutations
forbd = {}
forbdv = []
for x in forb:
    for t in Permutations(list(x)):
        c = vector(basis_coefs(t))
        if str(c) not in forbd:
            forbd[str(c)]=0
            if sum(c)>=0:
                forbdv.append(c)
print("forbidden without symmetries:",len(forbd))
###





fn = 0
for x in forb:
    norm2 = sum(t^2 for t in x)
    if norm2>fn:
        fn = norm2
print("square L2 norm of forbidden:",fn)


small = 2 #max abs value of a small coefficient
sc = []
for c in mrange_iter([range(-small,small+1) for i in range(n)]):
    if sum(c)>=0:
        sc.append(vector(c))
sc.sort(key=lambda i: sum(x*x for x in i)) #abs(x) in place of x*x seems a bit slower
sc = sc[1:] #removing zero

print("small:",len(sc))

order = [(-1)^i*(1/4+i/2)-1/4 for i in range(1000)] #[0,-1,1,-2,2,...]

def check_sublattice(basis,target=Infinity): 
    A = matrix(basis) #rows
    #A = A.LLL() #benchmark to see if helps
    A = A.transpose() #columns

#     Ad = abs(A.det())
#     if Ad==0 or Ad>=target:
#         return Infinity
        
    for coefs in sc:
        if str(A*coefs) in forbd:
            return Infinity

    Ai = A.inverse()
    Ai_norms = [sum(t^2 for t in Ai[i]) for i in range(n)]
    M = []
    Mp = 1
    Mps = 1
    for i in range(n):
        mi = floor(sqrt(fn*Ai_norms[i]))
        M.append(mi)
        Mp *= (2*mi+1)
        Mps *= (2*min(small,mi)+1)
        
    if max(M)>small: #otherwise already considered
        if Mp-Mps<len(forbd): #faster using A, otherwise A_inv
            for coefs in mrange_iter([order[0:(2*M[i]+1)] for i in range(n)]):
                if sum(coefs)>=0 and max(abs(x) for x in coefs)>small:
                    if str(A*vector(coefs)) in forbd:
                        return Infinity
        else: #check if A_inv times forbidden is not in Z^n
            for x in forbdv:
                y = Ai*x
                allint = True
                for t in y:
                    if t!=floor(t):
                        allint = False
                        break
                if allint:
                    return Infinity   
    
    return 0#Ad



#random/exhaustive sublattice search, skip to the cell starting with good for search using non-forbidden generating vectors of small norm
#each type of a matrix is the numbers on the diagonal
maxd = search_for #bound for diagonal entries
mindet = search_for #minimum determinant
maxdet = search_for #maximum determinant

def all_types(m,d1,d2,n):
    res = []
    if n==1:
        for k in range(d1,min(d2,m)+1):
            res.append([k])
        return res
    for k in range(1,min(d2,m)+1):
        prev = all_types(m,ceil(d1/k),floor(d2/k),n-1)
        for x in prev:
            res.append([k]+x)
    return res

types = all_types(maxd,mindet,maxdet,n)

print("types:",len(types))

def ind_max(t): #given type, lists maxima of indexes to define entries above the diagonal
    return [t[i]^i for i in range(1,n)]


allc = 0
for t in types:
    allc += prod(ind_max(t))
print("all cases:",allc)

from random import randint
def rand_ind(t): #given type, return random indexes
    maxs = ind_max(t)
    return [randint(0,maxs[i]-1) for i in range(n-1)]

def sublat_mat(t,ind): #given type and indexes, return sublattice basis
    res = matrix(n)
    for i in range(n):
        res[i,i] = t[i]
    for i in range(1,n):
        tt = -floor(t[i]/2)
        indc = ind[i-1]
        for j in range(i):
            res[j,i] = tt + (indc % t[i])
            indc = floor(indc / t[i])
    return res

def exh_search():
    tm = time()
    bestv = Infinity
    best = []
    allcnt = 0
    for tt in types:
        print("type:",tt,prod(tt))
        cnt = 0
        maxinds = ind_max(tt)
        cM = prod(maxinds)
        print("cases of this type:",cM)
        for i in range(cM):
            allcnt += 1
            if allcnt%(log_freq*threads)==0:
                print(allcnt,(time()-tm)/3600*(allc-allcnt)/(log_freq*threads))
                tm=time()
            if allcnt>=start_from and allcnt%threads==thread:
                inds = []
                l = i
                for j in range(n-1):
                    inds.append(l%maxinds[j])
                    l=floor(l/maxinds[j])
                mat = sublat_mat(tt,inds)
                cs = check_sublattice(mat)
                if cs==0:
                    cs = prod(tt)
                    if cs<bestv:
                        print("sublattice basis:")
                        print(mat)
                        bestv = cs
                        best = [mat]
                    elif cs==bestv:
                        best.append(mat)
        print("best so far:",len(best))
    print("all cases:",allcnt)
    print("best:",len(best))
    return best





best = exh_search()




very short: 5
short: 39
0.018368244171142578
forbidden: 31
forbidden without symmetries: 1984
square L2 norm of forbidden: 264
small: 1752
types: 175
all cases: 723027151
type: [1, 1, 1, 1, 135] 135
cases of this type: 332150625
100000 42.561363487552626
200000 72.65329944755135
300000 69.94641734346992
400000 70.01737849723492
500000 69.68041843674021
600000 73.21942864545599
700000 78.01255001057177
800000 71.23830662023839
900000 71.78080691027117
1000000 68.98345977511292
1100000 71.40345018156718
1200000 59.974596807423644
1300000 24.687776271772464
1400000 69.16510457709218
1500000 67.428444859515
1600000 69.0164463261966
1700000 69.52449891456398
1800000 68.2984883319102
1900000 75.12450606748429
2000000 71.54906422024152
2100000 68.38368139650262
2200000 70.27363042742357
2300000 71.24922716400012
2400000 72.28573996759677
2500000 21.31052306169143
2600000 72.94951515177927
2700000 90.1513332658964
2800000 86.21525344728
2900000 88.704420207889
3000000 87.3430839034711
3100000 

30200000 72.37415325356086
30300000 70.12088329877943
30400000 74.42045239651503
30500000 71.97853315594139
30600000 69.71920118246267
30700000 70.52493060030581
30800000 14.472325681252789
30900000 72.11793450457431
31000000 69.17710327999143
31100000 71.46941815821853
31200000 72.05889814483409
31300000 71.10726860697457
31400000 73.16788340609045
31500000 75.10606697132596
31600000 73.866877120481
31700000 77.73266551199384
31800000 26.30908424779502
31900000 78.50455961519314
32000000 75.2422626152814
32100000 81.16053157892968
32200000 71.0781222209108
32300000 31.94359618341908
32400000 79.13974949639173
32500000 76.27905160271085
32600000 75.88883588233581
32700000 73.25764181391474
32800000 81.3457804717131
32900000 75.36129810242484
33000000 72.56534220799364
33100000 70.45235923359505
33200000 57.23595007585745
33300000 38.312665275774215
33400000 71.50574647736721
33500000 71.28710478118596
33600000 73.31943286439115
33700000 74.13551476456686
33800000 74.19038301187099
3390

60700000 64.89442863116531
60800000 68.12748625834665
60900000 62.48980816293017
61000000 68.46478504371389
61100000 28.492590805837168
61200000 78.03625963856233
61300000 70.70275680487863
61400000 70.67148178764849
61500000 73.27205107764627
61600000 72.37425434877575
61700000 73.54529363102924
61800000 72.0403523740454
61900000 74.11071992107811
62000000 38.729723191796886
62100000 62.461612507921664
62200000 72.5472950440691
62300000 71.14001310593308
62400000 68.46499204554942
62500000 65.77331763273442
62600000 62.857611042255236
62700000 63.45563169785288
62800000 19.218475073395997
62900000 63.590985974795075
63000000 64.32003974038003
63100000 70.63699255357321
63200000 63.539459400449125
63300000 67.9931294300303
63400000 72.63807095241943
63500000 49.49994538316695
63600000 51.149596191758526
63700000 72.87356914918155
63800000 70.87539039065263
63900000 71.44862785498178
64000000 70.75803063927874
64100000 72.19283646521919
64200000 70.25698998133373
64300000 71.59380757183

91000000 61.70495020994891
91100000 64.02275032913504
91200000 64.06961629950764
91300000 68.49562005087493
91400000 68.60245227881218
91500000 65.79584008427912
91600000 66.22907712657495
91700000 50.87994440027029
91800000 34.1836536515339
91900000 59.60856854169895
92000000 61.0633684908304
92100000 64.91895937174999
92200000 60.267707167211526
92300000 20.47890552559348
92400000 56.98184287720567
92500000 63.76878032387296
92600000 58.65885470864781
92700000 58.43411208187324
92800000 44.85942121389909
92900000 40.109461548208934
93000000 63.48765113559396
93100000 63.987700829400616
93200000 63.081145780856886
93300000 64.61757829253837
93400000 65.11354426233164
93500000 66.09113893840313
93600000 67.0898981058865
93700000 64.51148013225193
93800000 64.03060618825107
93900000 67.82016577354128
94000000 61.764744385651944
94100000 64.23292459708132
94200000 39.592453859807065
94300000 45.809883341366614
94400000 59.71418262917818
94500000 61.777453536892814
94600000 56.89841466824

120500000 65.46508865527935
120600000 67.1304063732877
120700000 66.52689514924737
120800000 70.8890108291509
120900000 65.56858695186905
121000000 66.60105461785646
121100000 69.43129631821306
121200000 59.42343610129257
121300000 57.6946459963563
121400000 61.11369196499396
121500000 21.73191938225572
121600000 57.10509879298572
121700000 58.3507669797061
121800000 33.25614869563617
121900000 44.3321228731271
122000000 61.37959426302011
122100000 54.91897059091765
122200000 39.71592015233635
122300000 57.044332217819225
122400000 59.99034426487357
122500000 60.314550523386856
122600000 62.73951141635758
122700000 63.68448430898786
122800000 69.66137213828239
122900000 68.68190089079893
123000000 69.57669139052672
123100000 67.04296290002917
123200000 68.41721753543703
123300000 62.48922191894593
123400000 63.017081800985764
123500000 61.81720135696297
123600000 60.570724196280075
123700000 60.776210576599844
123800000 57.14471930077982
123900000 63.11355591842447
124000000 25.5708756

...

best so far: 0
type: [1, 1, 3, 5, 9] 135
cases of this type: 7381125
639700000 2.0408428052379866
639800000 1.8641434694598398
639900000 1.817611463718483
640000000 1.7685671158765672
640100000 1.8094310688729949
640200000 1.982141375087293
640300000 1.9168053181458133
640400000 1.7700523502773602
640500000 1.8058707791626298
640600000 1.758907838322896
640700000 1.7380107950232657
640800000 1.7321150472555804
640900000 1.9037808953119515
641000000 2.123687502555665
641100000 1.9200611469581246
641200000 1.7269016015248617
641300000 1.7609892884552785
641400000 1.6869899473683074
641500000 1.6040844787075816
641600000 1.667956223989427
641700000 1.7070115085008386
641800000 1.9158905851197658
641900000 1.997105229818848
642000000 1.8958645323201906
642100000 1.6786030634761828
642200000 1.6146390860899098
642300000 1.4911824137957945
642400000 1.5132299619580625
642500000 1.6356869022568115
642600000 1.7776231815010117
642700000 1.9274437207725106
642800000 1.8814997765695862
642900000

667100000 2.1769121663771225
667200000 2.3230937862582706
667300000 2.3124349630931023
667400000 2.470620732061398
667500000 2.4293071194780747
667600000 2.411976682318614
667700000 2.5275447824765664
best so far: 0
type: [1, 1, 5, 3, 9] 135
cases of this type: 4428675
667800000 1.6334609216384284
667900000 0.9834493886693836
668000000 0.9712263865868449
668100000 0.9912892266401171
668200000 0.9760050217701335
668300000 0.9963916695254008
668400000 0.9952819889068577
668500000 0.9780427423373957
668600000 0.9605936672548082
668700000 0.9798846808899212
668800000 1.030103501183474
668900000 0.9927088788149722
669000000 0.9764496161282982
669100000 0.9578939151582043
669200000 1.0175499521129834
669300000 1.0977894088693498
669400000 1.0956817511289936
669500000 0.9842071869772262
669600000 1.0052231518814319
669700000 0.9975123209395647
669800000 1.2744766709038569
669900000 1.329594150804422
670000000 1.1654653588069182
670100000 1.0725700329241574
670200000 1.15195107671696
670300000

691700000 1.2160846017673155
691800000 1.6628830850755245
691900000 2.041915908997331
692000000 1.2872139937632043
692100000 1.8117584432523985
692200000 1.7734656568968592
692300000 1.2377224371192357
692400000 2.209342054476733
692500000 1.5314917822102865
692600000 1.3333594401723827
692700000 2.1274827783611436
692800000 1.2283125410533329
692900000 1.6037010756283032
693000000 1.844425757312177
693100000 1.4958245712152658
693200000 1.7848439057440402
693300000 1.6595722765689254
693400000 1.333676600730753
693500000 1.9158000182862627
693600000 1.2992211938679932
693700000 1.3948834914346735
693800000 1.7149004835827477
693900000 1.3784375744634243
694000000 1.5720294179911374
694100000 1.666902200491056
694200000 1.3701248020275714
694300000 1.61354010795964
694400000 1.6487700311043343
694500000 1.4633766597332911
694600000 1.2880353766769308
694700000 1.6356876354276375
694800000 1.7864839728285444
694900000 1.4437444237061083
695000000 1.5655582560358823
695100000 1.574947913

713000000 0.12639040756701564
713100000 0.058075567543286744
best so far: 0
type: [1, 9, 1, 5, 3] 135
cases of this type: 91125
713200000 0.058809267732560636
best so far: 0
type: [1, 9, 1, 15, 1] 135
cases of this type: 30375
best so far: 0
type: [1, 9, 3, 1, 5] 135
cases of this type: 50625
best so far: 0
type: [1, 9, 3, 5, 1] 135
cases of this type: 10125
713300000 0.08670958714711204
best so far: 0
type: [1, 9, 5, 1, 3] 135
cases of this type: 18225
best so far: 0
type: [1, 9, 5, 3, 1] 135
cases of this type: 6075
best so far: 0
type: [1, 9, 15, 1, 1] 135
cases of this type: 2025
best so far: 0
type: [1, 15, 1, 1, 9] 135
cases of this type: 98415
713400000 0.07092699892844756
best so far: 0
type: [1, 15, 1, 3, 3] 135
cases of this type: 32805
best so far: 0
type: [1, 15, 1, 9, 1] 135
cases of this type: 10935
best so far: 0
type: [1, 15, 3, 1, 3] 135
cases of this type: 10935
best so far: 0
type: [1, 15, 3, 3, 1] 135
cases of this type: 3645
best so far: 0
type: [1, 15, 9, 1, 1] 13

best so far: 0
type: [9, 1, 1, 3, 5] 135
cases of this type: 16875
best so far: 0
type: [9, 1, 1, 5, 3] 135
cases of this type: 10125
best so far: 0
type: [9, 1, 1, 15, 1] 135
cases of this type: 3375
723000000 0.00022380770271023117
best so far: 0
type: [9, 1, 3, 1, 5] 135
cases of this type: 5625
best so far: 0
type: [9, 1, 3, 5, 1] 135
cases of this type: 1125
best so far: 0
type: [9, 1, 5, 1, 3] 135
cases of this type: 2025
best so far: 0
type: [9, 1, 5, 3, 1] 135
cases of this type: 675
best so far: 0
type: [9, 1, 15, 1, 1] 135
cases of this type: 225
best so far: 0
type: [9, 3, 1, 1, 5] 135
cases of this type: 1875
best so far: 0
type: [9, 3, 1, 5, 1] 135
cases of this type: 375
best so far: 0
type: [9, 3, 5, 1, 1] 135
cases of this type: 75
best so far: 0
type: [9, 5, 1, 1, 3] 135
cases of this type: 405
best so far: 0
type: [9, 5, 1, 3, 1] 135
cases of this type: 135
best so far: 0
type: [9, 5, 3, 1, 1] 135
cases of this type: 45
best so far: 0
type: [9, 15, 1, 1, 1] 135
cases 